In [5]:
import numpy as np

In [6]:
a = [[1,2,3,4],[5,6,7,8]]
a = np.array(a)
new_shape = (4,2)

b = np.reshape(a, new_shape)

print("a" , a)
print("b" , b)


a [[1 2 3 4]
 [5 6 7 8]]
b [[1 2]
 [3 4]
 [5 6]
 [7 8]]


In [ ]:
# reshape vs transpose
# reshape 只改变形状，不改变元素顺序；transpose 会交换轴：
print("reshape(4,2):\n", a.reshape(4, 2))
print("transpose:\n", a.T)

reshape(4,2):
 [[1 2]
 [3 4]
 [5 6]
 [7 8]]
transpose:
 [[1 5]
 [2 6]
 [3 7]
 [4 8]]


In [13]:
def is_invertible(a):
    if len(a) != len(a[0]):
        return false
    return np.linalg.det(a) != 0

def transform_matrix(A, T, S):
    if not is_invertible(T) or not is_invertible(S) :
        print("T or S is not invertible")
        return -1
    else:
        transformed_matrix = (np.linalg.inv(T) @ A) @ S
    return transformed_matrix

print(transform_matrix([[1, 2], [3, 4]], [[2, 0], [0, 2]], [[1, 1], [0, 1]]))

[[0.5 1.5]
 [1.5 3.5]]


In [3]:
features = [[0.5, 1.0], [-1.5, -2.0], [2.0, 1.5]]
labels = [0, 1, 0]
weights = [0.7, -0.4]
bias = -0.1

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
prob =sigmoid(np.dot(features,weights) + bias)
print(prob)
mse = np.round(np.mean((labels - prob) **2), 4)
print(mse)

[0.46257015 0.41338242 0.66818777]
0.3349


In [1]:
import numpy as np

In [18]:
features = [[1.0, 2.0], [2.0, 1.0], [-1.0, -2.0]]
labels = [1, 0, 0]
initial_weights = [0.1, -0.2]
initial_bias = 0.0
learning_rate = 0.1
epochs = 2

In [ ]:
features = np.array(features)
labels = np.array(labels)
weights = np.array(initial_weights)
bias = initial_bias
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)
n = len(labels)
mse_loss = []
for epoch in range(epochs):
    # foward
    z = np.dot(features,weights) + bias
    predict = sigmoid(z)
    # mse loss
    loss = np.mean((np.array(labels) - predict) ** 2)
    mse_loss.append(loss)
    #gradient descent
    dloss_dpred = 2 * (predict - labels) / n   # n
    dpred_dz = sigmoid_derivative(predict)     # n
    dz_dw = features                           # n * m
    dz_db = 1

    print(dloss_dpred)
    print(dpred_dz)
    grad_z = dloss_dpred * dpred_dz  # n * n =  n
    print(grad_z)
    grad_w = np.dot(grad_z, dz_dw)  # n * m
    print("grad_w",grad_w)
    grad_b = np.sum(grad_z) * dz_db 
    print("grad_b", grad_b)
    # update weights and bias
    weights -= learning_rate*grad_w
    bias -= learning_rate*grad_b

print("weights", weights)
print("bias", bias)
print("mse_loss", mse_loss)

[-0.38296168  0.33333333  0.38296168]
[0.24445831 0.25       0.24445831]
[-0.09361817  0.08333333  0.09361817]
grad_w [-0.02056966 -0.29113933]
grad_b 0.08333333333333334
[-0.37446407  0.33748221  0.37172584]
[0.24619359 0.24996127 0.24668354]
[-0.09219065  0.08435748  0.09169864]
grad_w [-0.01517434 -0.28342112]
grad_b 0.08386547041703868
weights [ 0.1035744  -0.14254396]
bias -0.016719880375037202
mse_loss [0.3033228034139421, 0.2942232621822798]


In [8]:
class Value:
	def __init__(self, data, _children=(), _op=''):
		self.data = data
		self.grad = 0
		self._backward = lambda: None
		self._prev = set(_children)
		self._op = _op
	def __repr__(self):
		return f"Value(data={self.data}, grad={self.grad}, prev={self._prev}, op={self._op})"

	def __add__(self, other):
		# Implement addition here
		other = other if isinstance(other, Value) else Value(other)
		out = Value(self.data + other.data, (self, other), '+')
		def _backward():
			self.grad +=out.grad
			other.grad += out.grad
		out._backward = _backward
		return out

	def __mul__(self, other):
		# Implement multiplication here
		other = other if isinstance(other, Value) else Value(other)
		out = Value(self.data * other.data, (self, other), '*')
		def _backward():
			self.grad += other.data * out.grad
			other.grad += self.data * out.grad
		out._backward = _backward
		return out

	def relu(self):
		out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

		def _backward():
			self.grad += (out.data > 0) * out.grad
		out._backward = _backward

		return out

	def backward(self):
		# topological order all of the children in the graph
		topo = []
		visited = set()
		def build_topo(v):
			if v not in visited:
				visited.add(v)
				for child in v._prev:
					build_topo(child)
				topo.append(v)
		build_topo(self)

		# go one variable at a time and apply the chain rule to get its gradient
		self.grad = 1
		for v in reversed(topo):
			v._backward()

In [9]:
a = Value(2.0)
b = Value(3.0)
c = a * b


c.backward()

print("a:", a)
print("b:", b) 
print("c:", c)

a = Value(2.0)
b = Value(3.0)
d = a + b
d.backward()
print("a:", a)
print("b:", b) 
print("d:", d)

a: Value(data=2.0, grad=3.0, prev=set(), op=)
b: Value(data=3.0, grad=2.0, prev=set(), op=)
c: Value(data=6.0, grad=1, prev={Value(data=3.0, grad=2.0, prev=set(), op=), Value(data=2.0, grad=3.0, prev=set(), op=)}, op=*)
a: Value(data=2.0, grad=1, prev=set(), op=)
b: Value(data=3.0, grad=1, prev=set(), op=)
d: Value(data=5.0, grad=1, prev={Value(data=3.0, grad=1, prev=set(), op=), Value(data=2.0, grad=1, prev=set(), op=)}, op=+)


In [34]:

import numpy as np
import copy
import math

# DO NOT CHANGE SEED
np.random.seed(42)

# DO NOT CHANGE LAYER CLASS
class Layer(object):

	def set_input_shape(self, shape):
	
		self.input_shape = shape

	def layer_name(self):
		return self.__class__.__name__

	def parameters(self):
		return 0

	def forward_pass(self, X, training):
		raise NotImplementedError()

	def backward_pass(self, accum_grad):
		raise NotImplementedError()

	def output_shape(self):
		raise NotImplementedError()

# Your task is to implement the Dense class based on the above structure
class Dense(Layer):
	def __init__(self, n_units, input_shape=None):
		self.layer_input = None
		self.input_shape = input_shape
		self.n_units = n_units
		self.trainable = True
		self.W = None
		self.w0 = None
		self.optimizer_w = None
		self.optimizer_w0 = None
	
	def initialize(self, optimizer):
		limit  = 1 / np.sqrt(self.input_shape)
		self.W = np.random.uniform(-limit, limit, (self.input_shape[0], self.n_units))
		self.w0 = np.zeros((1,self.n_units))
		self.optimizer_w = optimizer
		self.optimizer_w0 = optimizer

	def forward_pass(self,x):
		self.x = x
		return np.dot(x, self.W) + self.w0

	def backward_pass(self, accum_grad):
		grad_w = np.dot(self.x.T, accum_grad)
		print("grad_w:", grad_w)
		grad_b = np.sum(accum_grad, axis=0, keepdims =True)
		print("grad_b:", grad_b)
		grad_input = np.dot(accum_grad, self.W.T)
		print("grad_input:", grad_input)
		self.optimizer_w.update(self.W,grad_w)
		self.optimizer_w0.update(self.w0,grad_b)
		return grad_input

	def number_of_parameters(self):
		return np.prod(self.W.shape) + np.prod(self.w0.shape),self.W.shape,self.w0.shape
	
	def output_shape(self):
		return (self.input_shape, self.n_units)

	

In [35]:
# Initialize a Dense layer with 3 neurons and input shape (2,)
dense_layer = Dense(n_units=3, input_shape=(2,))

# Define a mock optimizer with a simple update rule
class MockOptimizer:
    def update(self, weights, grad):
        return weights - 0.01 * grad

optimizer = MockOptimizer()

# Initialize the Dense layer with the mock optimizer
dense_layer.initialize(optimizer)


In [36]:
# Perform a forward pass with sample input data
X = np.array([[1, 2]])
output = dense_layer.forward_pass(X)
print("Forward pass output:", output)

# Perform a backward pass with sample gradient
accum_grad = np.array([[0.1, 0.2, 0.3]])
back_output = dense_layer.backward_pass(accum_grad)
print("Backward pass output:", back_output)

Forward pass output: [[ 0.10162127 -0.33551992 -0.64490545]]
grad_w: [[0.1 0.2 0.3]
 [0.2 0.4 0.6]]
grad_b: [[0.1 0.2 0.3]]
grad_input: [[ 0.20816524 -0.22928937]]
Backward pass output: [[ 0.20816524 -0.22928937]]


In [33]:
input_shape=(2,)
input_shape[0]
para, w, w0 = dense_layer.number_of_parameters()
print(w)
print(w0)

(2, 3)
(1, 3)


In [44]:
X = np.array([[1, 2]])
y = np.array([1, 2])
z = X.T
print(X.shape)
print(y.shape)
print(z.shape)
print(z)
x = np.array([1.0, 2.0])
print(x.shape)
w1 = np.array([[1.0, 0.0], [0.0, 1.0]])
print(w1.shape)

(1, 2)
(2,)
(2, 1)
[[1]
 [2]]
(2,)
(2, 2)


In [40]:
s = 6
a = 2
c = s / a
print(c)

3.0


In [43]:
import numpy as np
import math
def compute_cross_entropy_loss(predicted_probs: np.ndarray, true_labels: np.ndarray, epsilon = 1e-15) -> float:
    n = len(true_labels)
    loss = 0.0
    for i in range(n):
        for j in range(len(true_labels[i])):
            if true_labels[i][j] == 1:
                loss -= math.log(predicted_probs[i][j] + 1e-12)
    if loss / n == -0.0:
        return 0.0
    return loss / n
pred = np.array([[1, 0, 0], [0, 1, 0]])
true = np.array([[1, 0, 0], [0, 1, 0]])
print(compute_cross_entropy_loss(pred, true))
print(round(compute_cross_entropy_loss(pred, true), 4))

-1.000088900581841e-12
-0.0


In [50]:
x = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
print(x.shape)
print(np.mean(x, axis=(0,1)))
print(np.mean(x))
print(np.mean(x, axis=(0)))

(2, 2, 3)
[5.5 6.5 7.5]
6.5
[[4. 5. 6.]
 [7. 8. 9.]]


In [ ]:
import numpy as np

class LSTM:
	def __init__(self, input_size, hidden_size):
		self.input_size = input_size
		self.hidden_size = hidden_size

		# Initialize weights and biases
		self.Wf = np.random.randn(hidden_size, input_size + hidden_size)
		self.Wi = np.random.randn(hidden_size, input_size + hidden_size)
		self.Wc = np.random.randn(hidden_size, input_size + hidden_size)
		self.Wo = np.random.randn(hidden_size, input_size + hidden_size)

		self.bf = np.zeros((hidden_size, 1))
		self.bi = np.zeros((hidden_size, 1))
		self.bc = np.zeros((hidden_size, 1))
		self.bo = np.zeros((hidden_size, 1))

	def _sigmoid(self, x):
		return 1 / (1 + np.exp(-x))
	
	def _tanh(self, x):
		return np.tanh(x)

	def forward(self, x, initial_hidden_state, initial_cell_state):
		"""
		Processes a sequence of inputs and returns the hidden states, final hidden state, and final cell state.
		"""
		c_prev = initial_cell_state
		h_prev = initial_hidden_state
		outputs = []
		seq_len = x.shape[0]    #  x (length, input_size)
		for t in range(seq_len):
			xt = x[t].reshape(-1, 1)  # xt (input_size, 1)
			combined = np.vstack((xt, h_prev)) # combined (input_size + hidden_size, 1)


			# Wf * [h_prev, x_t] + bf
			# W is (hidden_size, input_size + hidden_size)
			# bf is (hidden_size, 1)
			# f is (hidden_size, 1)
			f_t = self._sigmoid(np.dot(self.Wf, combined) + self.bf)
			
			i_t = self._sigmoid(np.dot(self.Wi, combined) + self.bi)

			c_tilde = self._tanh(np.dot(self.Wc, combined) + self.bc)

			o_t = self._sigmoid(np.dot(self.Wo, combined) + self.bo)

			c_prev = f_t * c_prev + i_t * c_tilde
			h_prev = o_t * self._tanh(c_prev)

			outputs.append(h_prev.copy())
		print(h_prev)
		return np.array(outputs), h_prev, c_prev



In [28]:
input_sequence = np.array([[0.1, 0.2], [0.3, 0.4]])
initial_hidden_state = np.zeros((2, 1))
initial_cell_state = np.zeros((2, 1))

lstm = LSTM(input_size=2, hidden_size=2)
lstm.Wf = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.Wi = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.Wc = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.Wo = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.bf = np.array([[0.1], [0.2]]) 
lstm.bi = np.array([[0.1], [0.2]]) 
lstm.bc = np.array([[0.1], [0.2]]) 
lstm.bo = np.array([[0.1], [0.2]])
outputs, final_h, final_c = lstm.forward(input_sequence, initial_hidden_state, initial_cell_state)

print(final_h)

[[0.1]
 [0.2]]
[[0.3]
 [0.4]]
[[0.10978204]
 [0.33859662]]
[[0.10978204]
 [0.33859662]]


In [16]:
input_sequence = np.array([[0.1, 0.2], [0.3, 0.4]])

In [21]:
print(input_sequence[1])
xt = input_sequence[1].reshape(-1, 1) 
print(xt)
initial_hidden_state = np.zeros((2, 1))
print(initial_hidden_state)
print(np.vstack((xt, initial_hidden_state)))



[0.3 0.4]
[[0.3]
 [0.4]]
[[0.]
 [0.]]
[[0.3]
 [0.4]
 [0. ]
 [0. ]]


In [33]:
import numpy as np

class LSTM:
	def __init__(self, input_size, hidden_size):
		"""
		Initializes the LSTM with random weights and zero biases.
		
		Args:
			input_size (int): Size of the input vector.
			hidden_size (int): Size of the hidden state and cell state.
		"""
		self.input_size = input_size
		self.hidden_size = hidden_size

		# Weight matrices for forget gate, input gate, candidate cell state, and output gate
		self.Wf = np.random.randn(hidden_size, input_size + hidden_size)
		self.Wi = np.random.randn(hidden_size, input_size + hidden_size)
		self.Wc = np.random.randn(hidden_size, input_size + hidden_size)
		self.Wo = np.random.randn(hidden_size, input_size + hidden_size)

		# Bias vectors for forget gate, input gate, candidate cell state, and output gate
		self.bf = np.zeros((hidden_size, 1))
		self.bi = np.zeros((hidden_size, 1))
		self.bc = np.zeros((hidden_size, 1))
		self.bo = np.zeros((hidden_size, 1))

	def forward(self, x, initial_hidden_state, initial_cell_state):
		"""
		Processes a sequence of inputs and returns the hidden states at each time step,
		as well as the final hidden state and cell state.
		
		Args:
			x (np.ndarray): Input sequence of shape (T, input_size), where T is the sequence length.
			initial_hidden_state (np.ndarray): Initial hidden state of shape (hidden_size, 1).
			initial_cell_state (np.ndarray): Initial cell state of shape (hidden_size, 1).
		
		Returns:
			tuple: (hidden_states, final_hidden_state, final_cell_state)
				- hidden_states: List of hidden states at each time step.
				- final_hidden_state: Final hidden state after processing the sequence.
				- final_cell_state: Final cell state after processing the sequence.
		"""
		T, _ = x.shape
		h = initial_hidden_state
		c = initial_cell_state
		hidden_states = []

		for t in range(T):
			xt = x[t].reshape(-1, 1)  # Current input at time step t

			# Concatenate input and previous hidden state
			combined = np.vstack((xt, h))

			# Forget gate
			ft = self.sigmoid(np.dot(self.Wf, combined) + self.bf)

			# Input gate
			it = self.sigmoid(np.dot(self.Wi, combined) + self.bi)

			# Candidate cell state
			ct_tilde = np.tanh(np.dot(self.Wc, combined) + self.bc)

			# Update cell state
			c = ft * c + it * ct_tilde
			print("c",c)
			# Output gate
			ot = self.sigmoid(np.dot(self.Wo, combined) + self.bo)

			# Update hidden state
			h = ot * np.tanh(c)

			# Store the hidden state
			hidden_states.append(h)

		hidden_states = np.hstack(hidden_states)  # Convert list to array
		return hidden_states, h, c

	@staticmethod
	def sigmoid(x):
		"""
		Sigmoid activation function.
		"""
		return 1 / (1 + np.exp(-x))

# Example usage
if __name__ == "__main__":
	input_sequence = np.array([[0.1, 0.2], [0.3, 0.4]])  # Shape: (T=2, input_size=2)
	initial_hidden_state = np.zeros((2, 1))  # Shape: (hidden_size=2, 1)
	initial_cell_state = np.zeros((2, 1))  # Shape: (hidden_size=2, 1)

	lstm = LSTM(input_size=2, hidden_size=2)

	# Set weights and biases for reproducibility (optional)
	lstm.Wf = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
	lstm.Wi = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
	lstm.Wc = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
	lstm.Wo = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
	lstm.bf = np.array([[0.1], [0.2]])
	lstm.bi = np.array([[0.1], [0.2]])
	lstm.bc = np.array([[0.1], [0.2]])
	lstm.bo = np.array([[0.1], [0.2]])

	outputs, final_h, final_c = lstm.forward(input_sequence, initial_hidden_state, initial_cell_state)
	print("Final hidden state:\n", final_h)
	print("Final cell state:\n", final_c)

c [[0.08001526]
 [0.20937158]]
c [[0.19591405]
 [0.5543745 ]]
Final hidden state:
 [[0.10978204]
 [0.33859662]]
Final cell state:
 [[0.19591405]
 [0.5543745 ]]


In [31]:
input_sequence = np.array([[0.1, 0.2], [0.3, 0.4]])
initial_hidden_state = np.zeros((2, 1))
initial_cell_state = np.zeros((2, 1))

lstm = LSTM(input_size=2, hidden_size=2)
lstm.Wf = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.Wi = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.Wc = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.Wo = np.array([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]]) 
lstm.bf = np.array([[0.1], [0.2]]) 
lstm.bi = np.array([[0.1], [0.2]]) 
lstm.bc = np.array([[0.1], [0.2]]) 
lstm.bo = np.array([[0.1], [0.2]])
outputs, final_h, final_c = lstm.forward(input_sequence, initial_hidden_state, initial_cell_state)

print(final_h)

[[0.10978204]
 [0.33859662]]


In [38]:
import numpy as np

def pos_encoding_elem(pos, d_model):
    pos_encode = np.zeros(d_model)
    for i in range(0,d_model,2):
        angle = pos / np.power(10000, (2*i)/d_model)
        pos_encode[i] = np.sin(angle)
        if i + 1< d_model:
            pos_encode[i + 1] = np.cos(angle)
    return pos_encode

# output (position, d_model)
def pos_encoding(position: int, d_model: int):
	# Your code here
    if position == 0 or d_model <= 0:
        return -1
    else:
        pos_encoding = np.zeros((position,d_model))
        for pos in range(position):
            pos_encoding[pos] = pos_encoding_elem(pos, d_model)
        return np.round(pos_encoding,4)

def _get_position_encoding_matrix(max_seq_len: int, d_model: int) -> np.ndarray:
    """
    Generates position encoding matrix using vectorized operations.
    
    Args:
        max_seq_len (int): Maximum sequence length.
        d_model (int): Dimension of the model/embedding.
    
    Returns:
        np.ndarray: Position encoding matrix of shape (max_seq_len, d_model).
    """
    # Create position and dimension indices
    positions = np.arange(max_seq_len)[:, np.newaxis]     # Shape: (max_seq_len, 1)
    dims = np.arange(0, d_model, 2)[np.newaxis, :]       # Shape: (1, d_model/2)
    
    # Calculate angles using broadcasting
    angles = positions / np.power(10000, (2 * dims) / d_model)  # Shape: (max_seq_len, d_model/2)
    
    # Apply sin and cos functions
    pos_encoding = np.empty((max_seq_len, d_model))
    pos_encoding[:, 0::2] = np.sin(angles)
    pos_encoding[:, 1::2] = np.cos(angles)
    
    return pos_encoding


In [40]:
print(pos_encoding(2, 8))
print(_get_position_encoding_matrix(2, 8))
print(_get_position_encoding_matrix(5, 16))

[[0.000e+00 1.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 0.000e+00
  1.000e+00]
 [8.415e-01 5.403e-01 1.000e-02 1.000e+00 1.000e-04 1.000e+00 0.000e+00
  1.000e+00]]
[[0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00]
 [8.41470985e-01 5.40302306e-01 9.99983333e-03 9.99950000e-01
  9.99999998e-05 9.99999995e-01 1.00000000e-06 1.00000000e+00]]
[[ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 8.41470985e-01  5.40302306e-01  9.98334166e-02  9.95004165e-01
   9.99983333e-03  9.99950000e-01  9.99999833e-04  9.99999500e-01
   9.99999998e-05  9.99999995e-01  1.00000000e-05  1.00000000e+00
   1.00000000e-06  1.00000000e+00  1.00000000e-07  1.00000000e+00]
 [ 9.09297427e-01 -4.16146837e-01  1.98669331e-0

In [65]:

from turtle import forward
import numpy as np
class SimpleRNN:
	def __init__(self, input_size, hidden_size, output_size):
		"""
		Initializes the RNN with random weights and zero biases.
		"""
		self.hidden_size = hidden_size
		self.W_xh = np.random.randn(hidden_size, input_size)*0.01
		self.W_hh = np.random.randn(hidden_size, hidden_size)*0.01
		self.W_hy = np.random.randn(output_size, hidden_size)*0.01
		self.b_h = np.zeros((hidden_size, 1))
		self.b_y = np.zeros((output_size, 1))

		self.input_size =  input_size
		self.h_out = []
		self.pred_out = []
		self.loss_t = []
		self.loss = 0.0
	def forward(self, x):
		"""
		Forward pass through the RNN for a given sequence of inputs.
		"""
		h = np.zeros((self.hidden_size, 1))
		self.h_out.append(h)
		for i in range(x.shape[0]):
			xi = x[i].reshape(-1,1)
			h = np.tanh(np.dot(self.W_xh, xi) + np.dot(self.W_hh, h) + self.b_h)
			self.h_out.append(h)
			output =  np.dot(self.W_hy , h) + self.b_y
			self.pred_out.append(output)
		return self.pred_out
	

	def backward(self, x, y, learning_rate):
		"""
		Backpropagation through time to adjust weights based on error gradient.
		"""
		dWx = np.zeros_like(self.W_xh)
		dWh = np.zeros_like(self.W_hh)
		db = np.zeros_like(self.W_hy)
		dV = np.zeros_like(self.b_h)
		dc = np.zeros_like(self.b_y)

		dht_next = np.zeros((self.hidden_size, 1))

			   
		# Loop backward through time steps
		for t in reversed(range(x.shape[0])):
			self.forward( x)
			yt_true = y[t].reshape(-1,1)
			error_t = self.pred_out[t] - yt_true
			self.loss += 0.5 * np.sum(error_t ** 2)
			self.loss_t.append(error_t)

			# Gradient from the output layer at time t
			dYt = error_t   # shape: (output_size, 1)
			# Gradients for output layer parameters
			print(dYt)
			print(self.h_out[t])
			dV += np.dot(dYt, self.h_out[t].T)
			dc += dYt

			# Backpropagate into the hidden state (from output and future time steps)
			dht = np.dot(self.b_h.T, dYt) + dht_next  # shape: (hidden_size, 1)
			
			# Derivative through tanh activation: tanh derivative = (1 - h^2)
			dtanh = (1 - self.h_out[t] ** 2) * dht  # shape: (hidden_size, 1)
			
			# Current input at time step t
			xt = x[t].reshape(-1, 1)  # shape: (input_size, 1)
			
			# Gradients with respect to Wx, Wh, and bias
			dWx += np.dot(dtanh, xt.T)
			dWh += np.dot(dtanh, self.h[t-1].T)
			db  += dtanh
			
			# Propagate the gradient to previous hidden state
			dht_next = np.dot(self.Wh.T, dtanh)
		
		# Update parameters using the provided learning rate
		self.W_hx -= learning_rate * dWx
		self.W_hh -= learning_rate * dWh
		self.W_hy  -= learning_rate * db
		self.b_h  -= learning_rate * dV
		self.b_y  -= learning_rate * dc
		
		return dWx, dWh, db, dV, dc



In [77]:
import numpy as np

def softmax(values):
	values = values - np.max(values, -1, keepdims = True)
	exp_x = np.exp(values)
	return exp_x / np.sum(exp_x, -1, keepdims = True)

# crystal_values (n, dimension)
def pattern_weaver(n, crystal_values, dimension):
	crystal_values = np.array(crystal_values)
	print(crystal_values.shape)
	# Compute similarity matrix using dot product:
	logits = np.dot(crystal_values, crystal_values.T)
	print(logits)
	# Compute attention scores using softmax along each row:
	attn = softmax(logits)
	print(attn)
	# For each crystal, compute the weighted sum of all crystals:
	weighted = np.dot(attn, crystal_values)
	return weighted

In [78]:
print(pattern_weaver(5, [4, 2, 7, 1, 9], 1))

(5,)
151
1.0
[4. 2. 7. 1. 9.]


In [ ]:
import numpy as np

def compute_qkv(X, W_q, W_k, W_v):
	Q = np.matmul(X, W_q)
	K = np.matmul(X, W_k)
	V = np.matmul(X, W_v)
	return Q,K,V

def split_heads(x, n_heads):
	"""Split the last dimension into (n_heads, head_dim)."""
	seq_len, d_model = x.shape
	head_dim = d_model // n_heads
	return x.reshape(seq_len, n_heads, head_dim).transpose(1, 0, 2)

def combine_heads(x):
	"""Combine the head dimensions back to the original dimension."""
	n_heads, seq_len, head_dim = x.shape
	return x.transpose(1, 0, 2).reshape(seq_len, n_heads * head_dim)

def scaled_dot_product_attention(q, k, v, mask=None):
	"""Calculate the attention weights and output."""
	d_k = q.shape[-1]
	scores = np.matmul(q, k.transpose(0, 2, 1)) / np.sqrt(d_k)
	
	if mask is not None:
		scores = scores + mask
	
	attention_weights = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
	attention_weights = attention_weights / np.sum(attention_weights, axis=-1, keepdims=True)
	
	output = np.matmul(attention_weights, v)
	return output

def multi_head_attention(q, k, v, n_heads=2, mask=None):
	"""Multi-head attention mechanism without batch dimension."""
	assert q.shape[-1] % n_heads == 0, "d_model must be divisible by n_heads"
	
	# Split into multiple heads
	q_split = split_heads(q, n_heads)  # [n_heads, seq_len, head_dim]
	k_split = split_heads(k, n_heads)
	v_split = split_heads(v, n_heads)
	
	# Compute attention for each head
	attention_output = scaled_dot_product_attention(q_split, k_split, v_split, mask)
	
	# Combine heads back together
	combined = combine_heads(attention_output)  # [seq_len, d_model]
	
	return combined


In [82]:
m, n = 6, 8 
n_heads = 2 
np.random.seed(42) 
X = np.arange(m*n).reshape(m,n) 
X = np.random.permutation(X.flatten()).reshape(m, n) 
W_q = np.random.randint(0,4,size=(n,n)) 
W_k = np.random.randint(0,5,size=(n,n)) 
W_v = np.random.randint(0,6,size=(n,n)) 
Q, K, V = compute_qkv(X, W_q, W_k, W_v) 
test = Q
# test multi-head attention
actual_output = multi_head_attention(Q, K, V, n_heads) 
print(actual_output)

[[[432 381 409 366]
  [336 429 420 288]]

 [[300 157 254 300]
  [178 338 180 183]]

 [[392 286 435 376]
  [267 481 377 298]]

 [[329 214 370 257]
  [199 405 315 307]]

 [[412 263 315 294]
  [213 389 298 324]]

 [[330 340 418 345]
  [320 443 370 279]]]
[[[570 504 491 531]
  [304 449 307 408]]

 [[357 344 315 311]
  [185 369 285 292]]

 [[622 434 417 529]
  [265 539 363 428]]

 [[427 284 284 307]
  [176 376 360 349]]

 [[473 411 400 491]
  [235 371 358 342]]

 [[701 418 350 553]
  [294 498 286 382]]]
[[[547 490 399 495]
  [485 439 645 393]]

 [[203 293 199 399]
  [220 163 279 174]]

 [[471 472 429 538]
  [377 450 531 362]]

 [[398 388 244 338]
  [224 385 413 295]]

 [[541 331 366 492]
  [475 367 423 422]]

 [[500 463 514 447]
  [403 500 556 496]]]
[[547. 490. 399. 495. 377. 450. 531. 362.]
 [547. 490. 399. 495. 377. 450. 531. 362.]
 [547. 490. 399. 495. 377. 450. 531. 362.]
 [547. 490. 399. 495. 377. 450. 531. 362.]
 [547. 490. 399. 495. 377. 450. 531. 362.]
 [547. 490. 399. 495. 377. 45

In [95]:
test = np.array([[1., 0.,1., 0.], [0., 1.,1., 0.],[0., 1.,1., 0.],[0., 1.,1., 0.]])
print(test.shape)
print(test.reshape(4, 2, 2))
print("hello-----------")
print(test.reshape(4, 2, 2).transpose(1, 0, 2))



(4, 4)
[[[1. 0.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]]

 [[0. 1.]
  [1. 0.]]]
hello-----------
[[[1. 0.]
  [0. 1.]
  [0. 1.]
  [0. 1.]]

 [[1. 0.]
  [1. 0.]
  [1. 0.]
  [1. 0.]]]


In [96]:
X = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9]]])

In [97]:
print(X.shape)

(1, 3, 3)


In [98]:
def solution(A, B):
    max_possible = min(A, B)
    left = 1
    right = max_possible
    best = 0
    
    while left <= right:
        mid = (left + right) // 2
        total_sticks = (A // mid) + (B // mid)
        if total_sticks >= 4:
            best = mid
            left = mid + 1
        else:
            right = mid - 1
    return best

In [99]:
print(solution(10,21))
print(solution(13,11))
print(solution(2,1))
print(solution(1,8))

7
5
0
1


In [100]:
def solution(A, B):
    max_possible = min(A, B)
    left = 1
    right = max_possible
    best = 0
    
    while left <= right:
        mid = (left + right) // 2
        total_sticks = (A // mid) + (B // mid)
        if total_sticks >= 4:
            best = mid
            left = mid + 1
        else:
            right = mid - 1
    return best

In [101]:
print(solution(1,8))

1


In [107]:
def solution(letters):
    # 如果 letters 不是字符串，则尝试将其转换为字符串
    #if not isinstance(letters, str):
    #    letters = ''.join(letters)
    
    count = 0
    for c in 'abcdefghijklmnopqrstuvwxyz':
        lower = c
        upper = c.upper()
        if lower in letters and upper in letters:
            last_lower = letters.rfind(lower)
            first_upper = letters.find(upper)
            if last_lower < first_upper:
                count += 1
    return count

In [111]:
def solution2(letters):
    """
    Counts the number of different letters that appear in both lowercase and uppercase,
    where all lowercase occurrences appear before any uppercase occurrence.

    Args:
        letters (str): The input string of English letters.

    Returns:
        int: The number of different letters fulfilling the conditions.
    """
    seen_lower = set()  # Set to track lowercase letters seen
    invalid_letters = set()  # Set to track letters that violate the condition
    valid_letters = set()  # Set to track valid letters

    for char in letters:
        if char.islower():
            # If it's a lowercase letter, add it to the seen_lower set
            if char not in invalid_letters:
                seen_lower.add(char)
        elif char.isupper():
            # If it's an uppercase letter
            lower_char = char.lower()
            if lower_char not in seen_lower:
                # If the corresponding lowercase letter has not been seen, it's invalid
                invalid_letters.add(lower_char)
                if lower_char in valid_letters:
                    valid_letters.remove(lower_char)  # Remove from valid if previously added
            else:
                # If the lowercase letter was seen before, it's valid
                if lower_char not in invalid_letters:
                    valid_letters.add(lower_char)

    # Return the count of valid letters
    return len(valid_letters)

In [112]:
letters = "aaAbcCABBc"
print(solution(letters))
print(solution2(letters))
letters = "xyzXYZabcABC"
print(solution(letters))
print(solution2(letters))
letters = "ABCabcAefG"
print(solution(letters))
print(solution2(letters))


2
3
6
6
0
0
